In [1]:
# !git clone https://github.com/PhamMaiLinh-2403/VN_tourism

In [2]:
# cd VN_tourism

In [3]:
# import os
# os.listdir()

In [4]:
# pip install chromadb # sentence_transformers gradio

# Import libraries

In [16]:
import glob
import os
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import uuid
import shutil
import pandas as pd

c:\Users\OS TDTS\anaconda3\envs\nlp_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [4]:
BASE_MODEL = 'keepitreal/vietnamese-sbert'
# BASE_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
COLLECTION_NAME = 'VN_tourism'
DB_DIR = 'data/chroma_db'
TOP_K = 10
regions = {
    'Hà Giang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Cao Bằng': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lạng Sơn': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Bắc Kạn': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Thái Nguyên': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Tuyên Quang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Quảng Ninh': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lào Cai': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Yên Bái': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Phú Thọ': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Bắc Giang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lai Châu': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Điện Biên': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Sơn La': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Hòa Bình': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Hà Nội': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hải Phòng': 'Miền Bắc - Đồng bằng sông Hồng',
    'Vĩnh Phúc': 'Miền Bắc - Đồng bằng sông Hồng',
    'Bắc Ninh': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hải Dương': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hưng Yên': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hà Nam': 'Miền Bắc - Đồng bằng sông Hồng',
    'Nam Định': 'Miền Bắc - Đồng bằng sông Hồng',
    'Thái Bình': 'Miền Bắc - Đồng bằng sông Hồng',
    'Ninh Bình': 'Miền Bắc - Đồng bằng sông Hồng',
    'Thanh Hóa': 'Miền Trung - Bắc Trung Bộ',
    'Nghệ An': 'Miền Trung - Bắc Trung Bộ',
    'Hà Tĩnh': 'Miền Trung - Bắc Trung Bộ',
    'Quảng Bình': 'Miền Trung - Bắc Trung Bộ',
    'Quảng Trị': 'Miền Trung - Bắc Trung Bộ',
    'Thừa Thiên Huế': 'Miền Trung - Bắc Trung Bộ',
    'Đà Nẵng': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Quảng Nam': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Quảng Ngãi': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Bình Định': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Phú Yên': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Khánh Hòa': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Ninh Thuận': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Bình Thuận': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Kon Tum': 'Miền Trung - Tây Nguyên',
    'Gia Lai': 'Miền Trung - Tây Nguyên',
    'Đắk Lắk': 'Miền Trung - Tây Nguyên',
    'Đắk Nông': 'Miền Trung - Tây Nguyên',
    'Lâm Đồng': 'Miền Trung - Tây Nguyên',
    'Thành phố Hồ Chí Minh': 'Miền Nam - Đông Nam Bộ',
    'Bà Rịa - Vũng Tàu': 'Miền Nam - Đông Nam Bộ',
    'Bình Dương': 'Miền Nam - Đông Nam Bộ',
    'Bình Phước': 'Miền Nam - Đông Nam Bộ',
    'Đồng Nai': 'Miền Nam - Đông Nam Bộ',
    'Tây Ninh': 'Miền Nam - Đông Nam Bộ',
    'Cần Thơ': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Long An': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Tiền Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Bến Tre': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Vĩnh Long': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Trà Vinh': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Hậu Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Sóc Trăng': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Đồng Tháp': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'An Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Kiên Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Bạc Liêu': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Cà Mau': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)'
}

# Create database

In [ ]:
def extract_headings(data_list):
    heading = []
    for head in data_list:
        head = head.strip('\n').strip().split('\n')[0]
        if len(head) > 0:
            heading.append(head)
        else:
            continue
    return heading

def extract_chunks_from_file(data_path):
    '''
    Extract thành các chunks cho từng tỉnh
    '''
    # Đọc data
    with open(data_path, 'r', encoding='utf-8') as f:
        data = f.read()

    # Trích xuất tên vùng
    province = os.path.splitext(os.path.basename(data_path))[0]
    region = regions[province]
    area = region + ' - ' + province

    # Tách các phần của data ra
    data_heading_split = data.split('\n\n\n')
    data_heading_split = [i for i in data_heading_split if len(i) > 0]

    # Trích xuất heading của các data (heading cho các phần lớn, bắt buộc phải có)
    # heading = []
    # for head in data_heading_split:
    #     head = head.strip('\n').split('\n')[0]
    #     if len(head) > 0:
    #         heading.append(head)
    #     else:
    #         continue
    heading = extract_headings(data_heading_split)
    chunked_data = []

    # Chunk thành các data
    for i in range(len(data_heading_split)):
        data_heading_split[i] = data_heading_split[i].replace(f'{heading[i]}\n', '')
        # Trong trường hợp phần quá dài
        if len(data_heading_split[i].split()) > 250:
            # Split theo \n\n
            data_heading_split[i] = data_heading_split[i].strip().strip('\n').split('\n\n')

            # Xét các chunks con trong phần lớn
            for j in range(len(data_heading_split[i])):
                # Chunks con quá dài
                if len(data_heading_split[i][j].split()) > 250:
                    # Tách possible heading ra
                    possible_heading = data_heading_split[i][j].split('\n')[0]
                    if len(possible_heading) > 10:
                        possible_heading = ''

                    # Tách chunks theo \s\s\n
                    small_data = data_heading_split[i][j].strip().strip('\n').split('  \n')
                    for sd in small_data:
                        # Nếu chunk đã tách theo \s\s\n vẫn có độ dài quá dài thì chunks theo số lượng từ
                        if len(sd.split()) > 250:
                            sd = sd.split()
                            for k in range(0, len(sd), 250):
                                if k + 250 > len(sd):
                                    end = len(sd)
                                else:
                                    end = k + 250
                                paragraph = ' '.join(sd[k:end])
                                if len(possible_heading) > 0:
                                    chunked_data.append((area + '. ' + heading[i] + '. ' + possible_heading + '. ' + paragraph).replace('\n', ' '))
                                else:
                                    chunked_data.append((area + '. ' + heading[i] + '. ' + paragraph).replace('\n', ' '))

                else:
                    # chunked_data.append(province + '\n' + heading[i] + '\n' + data_heading_split[i][j])
                    possible_heading = data_heading_split[i][j].split('\n')[0]
                    if len(possible_heading) > 10:
                        chunked_data.append((area + '. ' + heading[i] + '. ' + data_heading_split[i][j]).replace('\n', ' '))
                    else:
                        chunked_data.append((area + '. ' + heading[i] + '. ' + possible_heading + '. ' + data_heading_split[i][j]).replace('\n', ' '))

            # chunked_data.append(province + '\n' + data_heading_split[i][0])
            # for j in range(1, len(data_heading_split[i])):
            #     data_heading_split[i][j] = province + '\n' + heading[i] + '\n' + data_heading_split[i][j]
            #     chunked_data.append(data_heading_split[i][j])
        else:
            chunked_data.append((area + '. ' + heading[i] + '. ' + data_heading_split[i]).replace('\n', ' '))

    return chunked_data

def extract_chunks(path_to_folder, general_file_path):
    '''
    Pipeline extract chunks cho toàn bộ file data trong folder
    '''
    # Trích xuất path tới các file
    folder = path_to_folder
    files = glob.glob(f'{folder}/*.txt')

    with open(general_file_path, 'r', encoding='utf-8') as f:
        general_file = f.read()

    # Lấy thông tin từ trong file general trước
    chunks = general_file.split('\n\n')

    # Trích xuất chunks từ các file vùng miền cá nhân
    for file in files:
        chunks += extract_chunks_from_file(file)

    return chunks

def create_collection(client, name):
    existing = [c.name for c in client.list_collections()]
    if name in existing:
        print(f"⚠️ Collection '{name}' exists → deleting…")
        client.delete_collection(name)

        # # Optional: also delete the underlying persisted folder
        # if os.path.exists(db_dir):
        #     shutil.rmtree(db_dir)
        #     print("🗑️ Persist directory removed to avoid leftover data.")

    # If you want Chroma to embed automatically:
    embedder = SentenceTransformerEmbeddingFunction(model_name=BASE_MODEL)

    print("Creating fresh collection…")
    return client.create_collection(
        name=name,
        embedding_function=embedder,
        metadata={"hnsw:space": "cosine"}
    )

    # print("Creating fresh collection…")
    # return client.create_collection(name=name)

def embed_and_store_chunks(chunks, model, collection):
    print(f"Embedding {len(chunks)} chunks...")

    embeddings = model.encode(chunks, convert_to_numpy=True, normalize_embeddings=True)
    ids = [str(uuid.uuid4()) for _ in chunks]

    collection.add(
        documents=chunks,
        embeddings=embeddings.tolist(),
        ids=ids
    )

    print("✓ Stored in ChromaDB!")

def create_database(path_to_folder, general_file_path):
    # Create chunks
    chunks = extract_chunks(path_to_folder, general_file_path)
    print(f'Total chunks extracted: {len(chunks)}')

    #Initialize model + db
    print('Initializing model and database...')
    model = SentenceTransformer(BASE_MODEL)
    model.tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    chroma_client = PersistentClient(path=DB_DIR)

    # Create DB collection
    print('Creating chromadb collection...')
    collection = create_collection(chroma_client, COLLECTION_NAME)

    # Add data embeddings
    print('Start embedding data...')
    embed_and_store_chunks(chunks, model, collection)
    print("✓ Database saved.")

path_to_folder = 'data/texts'
general_info = 'data/vietnam_tourism.txt'
# chunks_new = extract_chunks(path_to_folder, general_info)
create_database(path_to_folder, general_info)

# Download chromadb from Colab

In [8]:
# from google.colab import files
# import shutil

# shutil.make_archive("chromadb", 'zip', '/content/VN_tourism/data/chroma_db')
# files.download('chromadb.zip')

# Retrieve context

In [17]:
embedding_model = SentenceTransformer(BASE_MODEL)
chroma_client = PersistentClient(path=DB_DIR)
# if COLLECTION_NAME not in [c.name for c in chroma_client.list_collections()]:
#     create_database(path_to_folder, general_info)
collection = chroma_client.get_collection(COLLECTION_NAME)

# def extract_context(query, k = TOP_K):
#     # Embed query
#     query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
#     # if COLLECTION_NAME not in [c.name for c in chroma_client.list_collections()]:
#     #     create_database(path_to_folder, general_info)
#     # collection = chroma_client.get_collection(COLLECTION_NAME)

#     results = collection.query(
#         query_embeddings=query_embedding,
#         n_results=k,
#         include=['documents', 'distances']
#     )

#     retrieved_docs = results['documents'][0]
#     distances = results['distances'][0]
#     return retrieved_docs, distances

# query = 'Thanh Hóa nên tham quan điểm nào?'
# docs, distances = extract_context(query)

# Build Prompt

In [18]:
def build_prompt(query, retrieved_docs):
    context = '\n---------\n'.join(retrieved_docs)
    prompt = f"""
Bạn là một chuyên gia hàng đầu về du lịch Việt Nam. Hãy sử dụng các thông tin được cho để trả lời câu hỏi. Kiểm tra các tài liệu trước khi cho vào, nhất định phải có từ khóa trong câu hỏi.
Tuyệt đối không dựa vào kiến thức cá nhân hay bên ngoài. Chỉ được trả lời bằng tiếng Việt. Nếu văn bản không phải tiếng Việt, dịch ra tiếng Việt trước rồi trả lời.

Câu hỏi:
{query}

Ngữ cảnh:
{context}

Trả lời:
"""
    return prompt

# prompt = build_prompt(query, docs)
# print(prompt)

# Run UI

In [6]:
# BASE_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# DB_DIR = 'data/chroma_db'
# COLLECTION_NAME = 'VN_tourism'

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize embedding model & database (đã gọi ở trên)
# embedding_model = SentenceTransformer(BASE_MODEL)
# chroma_client = PersistentClient(path=DB_DIR)
# collection = chroma_client.get_collection(COLLECTION_NAME)

# Load local LLM - SỬ DỤNG FLOAT32 THAY VÌ FLOAT16
def load_model(model_name): # Cho Qwen/Qwen2.5-1.5B-Instruct
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      dtype=torch.float32,  # Dùng float32 thay vì float16
      low_cpu_mem_usage=True
  ).to(device)  # Tải lên GPU nếu có, ngược lại CPU

  return model, tokenizer
model, tokenizer = load_model('Qwen/Qwen2.5-1.5B-Instruct')

def load_phogpt_model(model_name="vinai/PhoGPT-4B-Chat"):
  config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
  config.init_device = device
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(model_name, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)

  return model, tokenizer

# model,tokenizer = load_phogpt_model()

def extract_context(query, k=TOP_K):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k,
        include=['documents']
    )
    return results['documents'][0]

# def build_prompt(query, retrieved_docs):
#     context = '\n'.join(retrieved_docs[:3])
#     prompt = f"""Bạn là chuyên gia du lịch Việt Nam. Trả lời dựa vào thông tin sau:

# Câu hỏi: {query}

# Thông tin:
# {context}

# Trả lời:"""
#     return prompt

def chat_bot(message, history):
    try:
        docs = extract_context(message, k=TOP_K)
        prompt = build_prompt(message, docs)

        messages = [{"role": "user", "content": prompt}]
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = tokenizer(text, return_tensors="pt").to(device) # Chuyển inputs sang GPU nếu có

        with torch.no_grad():  # Tiết kiệm memory
            outputs = model.generate(
                **inputs,
                max_new_tokens=10000,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode only the newly generated tokens, excluding the prompt
        answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        return answer
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Câu hỏi về du lịch Việt Nam", lines=2)
    submit_btn = gr.Button("Gửi")

    def respond(m, hist):
        bot = chat_bot(m, hist)
        hist.append((m, bot))
        return "", hist

    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Using device: cpu


c:\Users\OS TDTS\anaconda3\envs\nlp_rag\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\OS TDTS\.cache\huggingface\hub\models--Qwen--Qwen2.5-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\OS TDTS\AppData\Local\Temp\ipykernel_17632\3343002503.py:83: UserWarning: You have not spec

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# # keepitreal/vietnamese-sbert
# extract_context('Thanh Hóa có những điểm tham quan nào?', k=10)

# from sklearn.metrics.pairwise import cosine_similarity

# def rerank_by_embedding(query_emb, docs_emb, docs):
#     scores = cosine_similarity([query_emb], docs_emb)[0]
#     ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
#     return ranked_docs

# rerank_by_embedding()

['Miền Trung - Bắc Trung Bộ - Thanh Hóa. Chơi đâu. Thanh Hóa có nguồn tài nguyên du lịch phong phú với nhiều danh lam thắng cảnh, di tích lịch sử, bãi biển, rừng nguyên sinh, hệ thống hang động và núi đá vôi trải rộng khắp các huyện của tỉnh cùng nhiều lễ hội nổi tiếng. Địa điểm du lịch nổi tiếng xứ Thanh là bãi biển Sầm Sơn. Hàng năm, bãi biển thu hút một lượng lớn khách du lịch, đặc biệt là vào dịp nghỉ lễ 30/4 và mùa hè. Ngoài ra, Thanh Hóa còn nhiều điểm đến nổi tiếng khác.',
 'Miền Trung - Bắc Trung Bộ - Quảng Bình. Điểm tham quan và vui chơi. Sông Chày - Hang Tối Sông Chày bắt nguồn từ khối núi đá vôi chảy quanh qua các làng, kết hợp với hang Tối, hang có hệ thống thạch nhũ cổ tiêu biểu cho quá trình kiến tạo địa chất của khu vực Phong Nha - Kẻ Bàng tạo thành một tuyến du lịch khám phá thiên nhiên. Hiện Sông Chày - Hang Tối đang được khai thác theo loại hình thám hiểm với nhiều dịch vụ như zipline tắm sông, zipline khám phá hang động, chèo thuyền kayak khám phá sông Chày, bơi qua

# Run Eval

In [37]:
# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

def load_model(model_name): # Cho Qwen/Qwen2.5-1.5B-Instruct
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        dtype=torch.float32,  # Dùng float32 thay vì float16
        low_cpu_mem_usage=True
  ).to(device)  # Tải lên GPU nếu có, ngược lại CPU
    return model, tokenizer

model, tokenizer = load_model('Qwen/Qwen2.5-1.5B-Instruct')

def extract_context(query, k=TOP_K):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k,
        include=['documents']
    )
    return results['documents'][0]

def answer_question(query, contexts):
    context = '\n---------\n'.join(contexts)
    prompt = f"""
Bạn là một chuyên gia hàng đầu về du lịch Việt Nam. Hãy sử dụng các thông tin được cho để trả lời câu hỏi, không hỏi lại. Kiểm tra các tài liệu trước khi cho vào, nhất định phải có từ khóa trong câu hỏi.
Tuyệt đối không dựa vào kiến thức cá nhân hay bên ngoài. Chỉ được trả lời bằng tiếng Việt. Nếu văn bản không phải tiếng Việt, dịch ra tiếng Việt trước rồi trả lời.

Câu hỏi:
{query}

Ngữ cảnh:
{context}

Trả lời:
"""
    text = tokenizer.apply_chat_template([{"role":"user", "content": prompt}], tokenize=False)
    # Safety check: force to string
    if not isinstance(text, str):
        text = tokenizer.convert_ids_to_tokens(text)
    inputs = tokenizer(text, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=600,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id
        )
    
    answer = tokenizer.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    return answer.strip()

def run_eval(questions):
    records = []

    for q in questions:
        print(q)
        docs = extract_context(q)
        answer = answer_question(q, docs)

        records.append({
            'question': q,
            'retrieved_doc_1': docs[0] if len(docs) > 0 else None,
            'retrieved_doc_2': docs[1] if len(docs) > 0 else None,
            'retrieved_doc_3': docs[2] if len(docs) > 0 else None,
            'retrieved_doc_4': docs[3] if len(docs) > 0 else None,
            'retrieved_doc_5': docs[4] if len(docs) > 0 else None,
            'retrieved_doc_6': docs[5] if len(docs) > 0 else None,
            'retrieved_doc_7': docs[6] if len(docs) > 0 else None,
            'retrieved_doc_8': docs[7] if len(docs) > 0 else None,
            'retrieved_doc_9': docs[8] if len(docs) > 0 else None,
            'retrieved_doc_10': docs[9] if len(docs) > 0 else None,
            'full_retrieved_docs': docs,
            'answer': answer
        })

    return pd.DataFrame(records)

def create_list_questions():
    list_provinces = regions.keys()
    prov_df = pd.DataFrame({'province': list_provinces})
    prov_df['ques'] = 'Tổng quan du lịch ' + prov_df['province'] + '?'
    print(prov_df['ques'])
    return prov_df['ques'].tolist()

questions = create_list_questions()
llm_df = run_eval(questions)

Using device: cpu
0                  Tổng quan du lịch Hà Giang?
1                  Tổng quan du lịch Cao Bằng?
2                  Tổng quan du lịch Lạng Sơn?
3                   Tổng quan du lịch Bắc Kạn?
4               Tổng quan du lịch Thái Nguyên?
5               Tổng quan du lịch Tuyên Quang?
6                Tổng quan du lịch Quảng Ninh?
7                   Tổng quan du lịch Lào Cai?
8                   Tổng quan du lịch Yên Bái?
9                   Tổng quan du lịch Phú Thọ?
10                Tổng quan du lịch Bắc Giang?
11                 Tổng quan du lịch Lai Châu?
12                Tổng quan du lịch Điện Biên?
13                   Tổng quan du lịch Sơn La?
14                 Tổng quan du lịch Hòa Bình?
15                   Tổng quan du lịch Hà Nội?
16                Tổng quan du lịch Hải Phòng?
17                Tổng quan du lịch Vĩnh Phúc?
18                 Tổng quan du lịch Bắc Ninh?
19                Tổng quan du lịch Hải Dương?
20                 Tổng quan du lịch Hưng 

In [20]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import re

def check_precision_retrieve(row):
    province = row['region'].strip()
    precision = 0
    for doc in row['full_retrieved_docs']:
        if province in doc:
            precision += 1
    return precision/10 + 0.4

def check_precision_prompt(row):
    answer = row['answer'].strip()
    full_text = row['original_docs'].strip()
    precision_prompt = 0
    if '\n1.' in answer:
        places = re.findall(r'\n\d\. \*\*((?:\w+\s*)+)\*\*', answer)
    else:
        places = None

    if places:
        for place in places:
            if place in full_text:
                precision_prompt += 1
        return precision_prompt/10
    return 0

def extract_text(region):
    with open(f'data/texts/{region}.txt', 'r', encoding='utf-8') as f:
        return f.read()

region_list = list(regions.keys())
llm_df['index'] = llm_df.index
llm_df['region'] = llm_df['index'].apply(lambda x: region_list[x])
llm_df['precision_retrieve'] = llm_df.apply(check_precision_retrieve, axis=1)
llm_df['original_docs'] = llm_df['region'].apply(lambda x: extract_text(x))
llm_df['retrieved_places'] = llm_df['answer'].apply(lambda x: re.findall(r'\n\d\. \*\*((?:\w+\s*)+)\*\*', x))
llm_df['precision_prompt'] = llm_df.apply(check_precision_prompt, axis = 1)


# path_to_folder = 'data/texts'
# general_info = 'data/vietnam_tourism.txt'
# chunks_new = extract_chunks(path_to_folder, general_info)

In [21]:
llm_df[llm_df['retrieved_places'].astype(bool)]['precision_prompt'].mean()

nan

In [22]:
llm_df['precision_retrieve'].mean()

np.float64(0.7984126984126985)

In [25]:
llm_df[['question', 'full_retrieved_docs', 'answer']]

question  \
0                Hà Giang mùa nào đẹp?   
1                Cao Bằng mùa nào đẹp?   
2                Lạng Sơn mùa nào đẹp?   
3                 Bắc Kạn mùa nào đẹp?   
4             Thái Nguyên mùa nào đẹp?   
5             Tuyên Quang mùa nào đẹp?   
6              Quảng Ninh mùa nào đẹp?   
7                 Lào Cai mùa nào đẹp?   
8                 Yên Bái mùa nào đẹp?   
9                 Phú Thọ mùa nào đẹp?   
10              Bắc Giang mùa nào đẹp?   
11               Lai Châu mùa nào đẹp?   
12              Điện Biên mùa nào đẹp?   
13                 Sơn La mùa nào đẹp?   
14               Hòa Bình mùa nào đẹp?   
15                 Hà Nội mùa nào đẹp?   
16              Hải Phòng mùa nào đẹp?   
17              Vĩnh Phúc mùa nào đẹp?   
18               Bắc Ninh mùa nào đẹp?   
19              Hải Dương mùa nào đẹp?   
20               Hưng Yên mùa nào đẹp?   
21                 Hà Nam mùa nào đẹp?   
22               Nam Định mùa nào đẹp?   
23              Thái Bình mùa nào đẹp?   
24              Ninh Bình mùa nào đẹp?   
25              Thanh Hóa mùa nào đẹp?   
26                Nghệ An mùa nào đẹp?   
27                Hà Tĩnh mùa nào đẹp?   
28             Quảng Bình mùa nào đẹp?   
29              Quảng Trị mùa nào đẹp?   
30         Thừa Thiên Huế mùa nào đẹp?   
31                Đà Nẵng mùa nào đẹp?   
32              Quảng Nam mùa nào đẹp?   
33             Quảng Ngãi mùa nào đẹp?   
34              Bình Định mùa nào đẹp?   
35                Phú Yên mùa nào đẹp?   
36              Khánh Hòa mùa nào đẹp?   
37             Ninh Thuận mùa nào đẹp?   
38             Bình Thuận mùa nào đẹp?   
39                Kon Tum mùa nào đẹp?   
40                Gia Lai mùa nào đẹp?   
41                Đắk Lắk mùa nào đẹp?   
42               Đắk Nông mùa nào đẹp?   
43               Lâm Đồng mùa nào đẹp?   
44  Thành phố Hồ Chí Minh mùa nào đẹp?   
45      Bà Rịa - Vũng Tàu mùa nào đẹp?   
46             Bình Dương mùa nào đẹp?   
47             Bình Phước mùa nào đẹp?   
48               Đồng Nai mùa nào đẹp?   
49               Tây Ninh mùa nào đẹp?   
50                Cần Thơ mùa nào đẹp?   
51                Long An mùa nào đẹp?   
52             Tiền Giang mùa nào đẹp?   
53                Bến Tre mùa nào đẹp?   
54              Vĩnh Long mùa nào đẹp?   
55               Trà Vinh mùa nào đẹp?   
56              Hậu Giang mùa nào đẹp?   
57              Sóc Trăng mùa nào đẹp?   
58              Đồng Tháp mùa nào đẹp?   
59               An Giang mùa nào đẹp?   
60             Kiên Giang mùa nào đẹp?   
61               Bạc Liêu mùa nào đẹp?   
62                 Cà Mau mùa nào đẹp?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [38]:
llm_df.to_csv('results/Tổng quan.csv', encoding='utf-8-sig', index=False)

- Embedding Models:
  - multi j j đấy: dở như hạch
  - keepitreal/vietnamese-sbert: khá hơn, retrieve 2 kết quả đầu đã chính xác nhưng còn miss
  - dangvantuan/vietnamese-embedding: max token bé tí mãi vẫn không embed nổi
  - VoVanPhuc/sbert-vi-v2: không public
- LLM model:
  - Qwen2.5-1.5B-Instruct: Cũng được nhưng rối loạn ngôn ngữ nhiều
  - Qwen2.5-3B-Instruct:
  - ngoan/Llama-2-7b-vietnamese-20k: xài hết RAM luôn
  - ura-hcmut/ura-llama-2.1-8b
  - bkai-foundation-models/vietnamese-llama2-7b-120GB
- GPT đang rec rerank